In [1]:
using DataFrames
using CSV

In [46]:
tornado_transactions_df = CSV.read("../data/tornado_transactions.csv", DataFrame)
tornado_traces_df = CSV.read("../data/tornado_traces.csv", DataFrame)

withdraw_transactions_df = CSV.read("../data/tornado_withdraw_df.csv", DataFrame)
deposit_transactions_df = CSV.read("../data/deposit_transactions.csv", DataFrame)

tornado_addresses = CSV.read("../data/tornadocontracts_abi.csv", DataFrame, header=["Address", "Currency", "3", "4","abi"])["Address"];

In [40]:
ENV["LINES"]=10
ENV["COLUMNS"]=10000

10000

Lets first look at the data obtained using the following querys: 

SELECT * FROM `bigquery-public-data.crypto_ethereum.transactions` WHERE `to_address` 
IN (SELECT `address` FROM `tornado_cash_transactions.tornadocontracts`) AND SUBSTR(`input`, 1, 10) = "0x21a0adb6";

SELECT * FROM `bigquery-public-data.crypto_ethereum.transactions` WHERE `to_address` 
IN (SELECT `address` FROM `tornado_cash_transactions.tornadocontracts`) AND SUBSTR(`input`, 1, 10) = "0xb214faa5";


In [9]:
deposit_transactions_df

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price
,String,Int64,Int64,String,String,Float64,Int64,Int64,String,Int64,Int64,Missing,Missing,Int64,String,Int64,String,Int64?,Int64?,Int64?,Int64
1,0xcf97c470a56d96625c7240d3004ae2abd9141d7ffc4383ab6f29a181e3562e8b,4,10,0xb050dec5a9010f8b77a3962369b7bc737d3ed4a5,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0.0,1200000,56000000000,0xb214faa5083d52f2c4e7a95e340a999805a8328b6546949ef4500ffbbd977845eade62bd,1677906,1048832,missing,missing,1,2020-11-02 17:47:30 UTC,11179130,0x21d86cba454fea4f7e43c68763d4cffec101b614554635a3f15d538049463d4f,missing,missing,missing,56000000000
2,0x2fadc4d54bf6e3ec5035f14c0096befe92999866309893e0eeaecdef955c0323,22,118,0x6e92bc493c6abbdd6a1b18416f003de2c873ab50,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0.0,1200000,63000000000,0xb214faa502843565a5499c26b04c7a9069f1ad2621aa1d8dce61480782b81d0c1c919489,8209080,1048832,missing,missing,1,2021-01-02 23:37:35 UTC,11578054,0xd52ababd1dbbbe50277c54473c881f47fd95e78081a349670d4988644e4d37c4,missing,missing,missing,63000000000
3,0x7f851ba1d7292ca565961073a1111f7408760540c6f94e479c72a2ae05399cf4,0,84,0x50b9d4af009b038506d4d84b035c451d1a3a20bc,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,1.0e17,800000,5000000000,0xb214faa51174c586ebae02a3b4f54a0ba6a9b7aba4d79ea9078c4a017a7ddb3d91150021,8465851,800000,missing,missing,0,2020-01-05 05:19:39 UTC,9218451,0x77e0fc60906714a54697cfd9275b622ad9d2d0271c4ec816b10fa3a09b103680,missing,missing,missing,5000000000
4,0xbd83053f8afa7777f54a4aca6b8e112fa31b888922dc5b9a9a65eb66e9a6996f,7,63,0x6c6e4816ecfa4481472ff88f32a3e00f2eaa95a1,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,1.0e17,800000,30838446643,0xb214faa527a20ba920c8ae877d67ce1ebd7420dafb3150e001eca78166fd6d66a5fd253e,6222489,800000,missing,missing,0,2020-05-27 03:30:44 UTC,10145408,0x837b3482443f027f6f045644bf002243f72304686015a2d6265676b2a2fc630b,missing,missing,missing,30838446643
5,0x7baf0a76f35c1dece97fff883aa7174454bed460b1bade05844080017170fc6c,240,171,0x8c4c44fd06f7f98f08bf6a9ca156cec9ee1f31f8,0xfd8610d20aa15b7b2e3be39b396a1bc3516c7144,0.0,800000,105000000000,0xb214faa50afc4880238aac4198bceddea0fbf85e9ad3de74dd20eae39f369b060b50f516,10723996,800000,missing,missing,0,2021-01-06 19:04:40 UTC,11602841,0x7c5f21ea2a92f5182ce8648f152b6fb3b4379096309dc2ba232ac663c4a0d1d7,missing,missing,missing,105000000000
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮


Here we have the first inconsistency. If you go to tornado cash stats page, you will find that the number of deposits is around 92.000
https://dune.xyz/poma/tornado-cash_1

Lets look at the withdrawls now:

In [12]:
withdraw_transactions_df

,Column1,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,recipient_address
,Int64,String,Int64,Int64,String,String,Int64,Int64,Int64,String,Int64,Int64,Missing,Missing,Int64,String,Int64,String,Missing,Missing,Missing,Int64,String
1,0,0x8211d6815cdd607a1f8a0d5db08b3cd0121d6df4c6f5b52023788883e6fe7a97,0,54,0x00058bf70d45aa62a493e5fc596974edde903324,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,0,421760,37000000000,0x21a0adb600000000000000000000000000000000000000000000000000000000000000e0262c5b75219fb59b3a6e535d04d7f4ddf7b8ffd3236263babebaf0e352717db22782f28dda1bfccf8b1c59d0595a400214a32bdad8dcd3b1753a4013a8d6555300000000000000000000000000058bf70d45aa62a493e5fc596974edde90332400000000000000000000000000058bf70d45aa62a493e5fc596974edde9033240000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000055def7d206642ea94235420978ff0ea81fd15e7d8d7f9ab2b261bf980bb1d321e78994d9d0c49408a22a7481681f7945d8be26efc23e183a412169da7af4422fb173319129d37b7db5df1dbb9f6f5db60f1e5f81ab07f4f030dac1ce83999e2cba949a94b22d1f4af9749f46306db16ea555cea7412951222f7fc5716eae5d0a358bd5d1527136137ec436cb67149a2ca998342359f4f2bc8904d1ae5b520522b4b6dbcdfec5217e30fcd60797cf601bd77b3b0e9157e2e2331408b2553a340e4c89700723498c6e30c1ca67a9a5d0e64a3013845f26299969e7c824aacf6e22ac16d06453342296443d0c7a37bea887ff98cb0888481525a0cbc761754d9d,2727540,321760,missing,missing,1,2020-07-16 04:21:43 UTC,10468347,0x97002c5d89010843864301297092eb60b3d902f1850d863717db7ec768e7896e,missing,missing,missing,37000000000,0x00058BF70d45AA62A493E5FC596974EdDe903324
2,1,0xcc8746c16a2bf379e8319b7c190676b77508f09f7a9cec13dd5f77514f446034,800,56,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,0,423207,13000000000,0x21a0adb600000000000000000000000000000000000000000000000000000000000000e00a52cba850f4ad215648a969675fe4f4b02d6ddae38a189e81396e742962928527e852fdad1aed9326a692b63973cf72c5d6b4a13a6c54599780e870a7c304c80000000000000000000000000039f22efb07a647557c7c5d17854cfd6d489ef30000000000000000000000000039f22efb07a647557c7c5d17854cfd6d489ef300000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010022b904e7729cc7af2d50170a9b57934740815876a1f96a43c832ece57ce95dae2d98b39685cb87a18d38e29c20271506d7103bc391b490a7d524c76c91ff4f9518442626bafb9d4d95c507fa0d3b4a19f34c457bb22dab65432c7c1cd90742200323a69be5b17c37f876c8c212625f955e69151f9b5f695899743a9798f11959226c0dfbd8cf0db70ac928850726479a0a741e0b26ad81fe1d4415d22ce258732c1a9adf37aa06373fea66b14dad706bd08c951d497b3ef8ca5c550f942cc726140dbbb541564c78a4daa44bd5ec69b192b8acf8df2ed2d86386b17abfbf991d1405c2269265226205195dd90d96119383d494b261c31e19b7b6de46ffcf9c1b,3032779,323207,missing,missing,1,2019-12-16 20:06:53 UTC,9117186,0x8dc6ade4626b61074d088ba600ddcc2686bdc1795681440ce9a86cc6dbdc4dac,missing,missing,missing,13000000000,0x0039F22efB07A647557C7C5d17854CFD6D489eF3
3,2,0x9bb7303af6ce69085abc3d9f4f5b7884a90023fd6e5925cb6ffed9737ebff78c,799,51,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,0,423207,13000000000,0x21a0adb600000000000000000000000000000000000000000000000000000000000000e00a52cba850f4ad215648a969675fe4f4b02d6ddae38a189e81396e742962928525ff03d81abc80888bb6e8bd553e9c47e8e765bd4a0ea9e96dc179ea0d2aad4f0000000000000000000000000039f22efb07a647557c7c5d17854cfd6d489ef30000000000000000000000000039f22efb07a647557c7c5d17854cfd6d489ef3000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [14]:
sorted_withdawals = sort!(withdraw_transactions_df, :block_timestamp, rev=true)

,Column1,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,recipient_address
,Int64,String,Int64,Int64,String,String,Int64,Int64,Int64,String,Int64,Int64,Missing,Missing,Int64,String,Int64,String,Missing,Missing,Missing,Int64,String
1,17633,0x25aaa01bcefc2632525456e96c1048cdaeea32ddd12ad09c9d610ce3455a3b16,4,60,0xb5ddf5840506883467e854d48a2b95b62f021eda,0x169ad27a470d064dede56a2d3ff727986b15d52b,0,551039,8000000000,0x21a0adb600000000000000000000000000000000000000000000000000000000000000e00cee606abda2ee3409d2b6950b155ea3bb57e6c2c402aa5a3fae8e4e7801a4d906750e7e4e7798ce22984cc4d42e7b7100052ae3d8408b8243b9b6e2a349efbc000000000000000000000000c4ba2f8ae11a427693102e43f2d7be11ce129f54000000000000000000000000c4ba2f8ae11a427693102e43f2d7be11ce129f54000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100051a5cf4288e9073738a2bb2a9ed3c6540929d882720c29e6858b18430d4365a067a28da596538255a21d7e286d8f9a5ea7f72f1d0aefc59d7f14b7ec92dd2241c0253036ef4b4ee5f8c17b394952d62672038c9e5859fb0231292250bed4ae725ae4973134a6d72822192f45dc1534c729133deec202211a7a89207c8645a3207721cc51f115aa047be43970bfe3e1b372f33265c3029b91bf50c5bb5bee94d079e86cd567e745bd1f33f16743dbe45b89e98a3514f6791006fcdd77ddf8ab20663ccc13c23f31ad9ce3b3aa1474799d1f9b67362617fc39c2a2f3f7b9483502cada37a0f349008f237144b9bd57dfe84582049a33ef318f294c7c0329e8f00,13128914,411628,missing,missing,1,2021-06-10 22:00:48 UTC,12609363,0x26dc10ecdfed366c0ee1e09bb978f8d4ae7d39d12ef413305b8c66570e2304ca,missing,missing,missing,8000000000,0xc4Ba2f8aE11A427693102e43F2D7BE11ce129F54
2,17192,0x2c878be157f7eb268e2e052d566f6499735a85344079be8e547cd7ce07353562,0,52,0xa9b079d27e5992d04935ba661970f8089546db0c,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,0,421772,13000000000,0x21a0adb600000000000000000000000000000000000000000000000000000000000000e020d7f99e545bc36fccd10cd4bd96f72d8dcc689149d59b73601d445b212a6470247878f0994800884cc5154dbef1b376a4107814f4667447c6427686270c51d0000000000000000000000000a9b079d27e5992d04935ba661970f8089546db0c000000000000000000000000a9b079d27e5992d04935ba661970f8089546db0c0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001002398a0ecaddb5c6a8f4d27599d61051615cc9855105c4da39916cf39bf8fc9581e2c0f61d73844447e507696563c13864619b123ad5b2df1b4ce58aa75b609f720753e400cd415abcd7894273d07a2cee6361ecfd0f4267f7af46008afec26902dbd521f24815d139daa88736fdc0d1b0d0240aa2c27cef275956b41c6c3041512479ec6d609eb83e3a3cda5317b53d9cc5fc423864a9a34dcbbf06f70273ebc29c4fa150d1567740dbb4a6ee45382f2d6c529fbffff08dab24ce0abbbb3f8ce0085f2f8a2ef2326fd71a36e490edaf33e1aea8215f79b768f2c676acb8340341849d8f1a8398b08ddb9fd8d7ccc53050e4c0354758550426bd51651dbfce617,5926371,273882,missing,missing,0,2021-06-05 08:10:20 UTC,12573327,0xf06eff8cc2a9a9e9c0104292a37e9d2cc866cf220446d8e6138356366b9b7965,missing,missing,missing,13000000000,0xA9B079D27E5992d04935Ba661970F8089546db0C
3,3305,0x12265a26081e9847549287523d17225ec514ea658e4e13177a3fcb70e67ea914,0,173,0x12902b10c44c6e1788061840a41a14b80d152379,0xa160cdab225685da1d56aa342ad8841c3b53f291,0,476209,55000000000,0x21a0adb600000000000000000000000000000000000000000000000000000000000000e000cb6b1b1ade6777d9152ef4d678a709ec5f7bb487152dcc6f0f361b048da32a285655efd38282e3030d2f75a67be5184e0e1875f1647dc33d1b384ab19c965600000000000000000000000012902b10c44c6e1788061840a41a14b80d15237900000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

So, it seems that we have withdrwals up to 2021-06-10. Again, this doesn't make sense.

As [this post](https://medium.com/@theimperialduke/ethereum-traces-not-transactions-3f0533d26aa) explain, in order to recalculate eth balances we need to use ethereum **traces** (check that the guys from dune also use eth.traces). Lets see:

Runing the same query as before but on the traces table:

SELECT * FROM `bigquery-public-data.crypto_ethereum.traces` WHERE `to_address` 
IN (SELECT `address` FROM `tornado_cash_transactions.tornadocontracts`) AND SUBSTR(`input`, 1, 10) IN ("0xb214faa5", "0x21a0adb6");

In [15]:
tornado_traces_df

,transaction_hash,transaction_index,from_address,to_address,value,input,output,trace_type,call_type,reward_type,gas,gas_used,subtraces,trace_address,error,status,block_timestamp,block_number,block_hash,trace_id
,String,Int64,String,String,Float64,String,String?,String,String,Missing,Int64,Int64?,Int64,String?,String?,Int64,String,Int64,String,String
1,0x14a3923eb7f30311240d6cbe838ac7e0639c309017395b36b130938d9894787c,48,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0xb214faa50e50cde66fa6f1227cab1eeebf0aff07940d861fa4b66d3d0b4c85ce6c76bf7f,0x,call,call,missing,990735,912248,41,2,missing,1,2021-05-22 16:22:08 UTC,12485152,0xe6f35f8a11022e6b42818fff43b16690c7fcb88a7931960f7c22cc35104ebf4b,call_0x14a3923eb7f30311240d6cbe838ac7e0639c309017395b36b130938d9894787c_2
2,0xc2c9732f7ea93698d46bafa3dbe0f6febcf799945ce016fe779dd7faa2fdc87f,82,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,1.21151e17,0x21a0adb600000000000000000000000000000000000000000000000000000000000000e015d3d1d30de13ba53da140ef7120f92c7f2b7952a336d34d9db51097dc44bfc80cf98273476befb8881206a0a48af6c63dad87d7960d862d5ad886e615706a9e0000000000000000000000002fe8e10633eae71b503c60180c4190710a132237000000000000000000000000be4d1e137a24af091be80ae58d652279665e3a27000000000000000000000000000000000000000000000000000002f30d20eecb00000000000000000000000000000000000000000000000001ae69bce4dc680000000000000000000000000000000000000000000000000000000000000001000ca661773a0664c7314927e92525a32f0631194b1e73a2873b1bdff4b5b01da82e35c99e3602de30071f80fae1b55170f1c0ab03e505759fedcf2c60d84c53781205853b85e25db939e5f1f429a8ef5a1b58fae8cf10e5e1ef24f671a0743372020923e0bd54bb4ca70ae777397cbb959cb4968c0125512534c77c226de85e2c23e77641570d2dd8a46cd52438dcc0c91ec54471e6ab5f165bb76f9a598828870387d21cd35e5ed1ff4bdb1719577faacdb01ccbafaf9c16726a981d86def97b19808cb12a6b727b65dee94edf7984639cb2986e8a72da0245f137eeac5aff4603f36e0bd6033055c04d4e6805cbec2fc4b6724391a146f10a551921e3161f9a,0x,call,call,missing,500758,447979,4,0,missing,1,2021-10-15 17:05:13 UTC,13423878,0x47ed0032aee2cafd9acdf32317c434f07b537b8400f836e9186687122dd7548c,call_0xc2c9732f7ea93698d46bafa3dbe0f6febcf799945ce016fe779dd7faa2fdc87f_0
3,0xcb7e6a27cd0e9bf71054b61982b20d12e98885667ee61447d02f7ae1de9ce671,36,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0xb214faa52c7747bcfca9ce0281515638387453bcc49c30f4f0331229e8540ecf2fa59c0a,0x,call,call,missing,1317149,941658,41,2,missing,1,2021-04-26 09:49:49 UTC,12315198,0x067f762a302cbced13ff6165da4d309d646e3abbc7863cc56f98434d5624f8cb,call_0xcb7e6a27cd0e9bf71054b61982b20d12e98885667ee61447d02f7ae1de9ce671_2
4,0xade7cb15e967438c42a9eeede86245195591a4d7f12f327f18231eaa3802ed93,68,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0xb214faa5062e06017fab545360bc7c68684c339acc2c05a16cda23ba05074286ebdb0d3a,0x,call,call,missing,1038629,958810,41,2,missing,1,2021-04-15 21:06:02 UTC,12247016,0xd7ce8779f739c346e2e97e8b3a6f7e51896f3783ba99c1e9a23f84f8d8d21bce,call_0xade7cb15e967438c42a9eeede86245195591a4d7f12f327f18231eaa3802ed93_2
5,0x810b236b3524ec07e464db6da2befd0c70550c312422d605ad38caea2f1dbbe9,93,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0xb214faa5268ebc52aa1ede130d959fc68d276bae6a13376f6ae637ab3330a433af3ef9ec,0x,call,call,missing,990760,912274,41,2,missing,1,2021-05-22 16:10:38 UTC,12485099,0xdd0e09ffa6d2b1e917f4e575c1bcd4e5ff22602b4c7862fdc93719871679eed9,call_0x810b236b3524ec07e464db6da2befd0c70550c312422d605ad38caea2f1dbbe9_2
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮


In [48]:
filter(row -> row.to_address ∉ tornado_addresses, tornado_traces_df) #Every row in this df is interacting with a tcash contract

,transaction_hash,transaction_index,from_address,to_address,value,input,output,trace_type,call_type,reward_type,gas,gas_used,subtraces,trace_address,error,status,block_timestamp,block_number,block_hash,trace_id
,String,Int64,String,String,Float64,String,String?,String,String,Missing,Int64,Int64?,Int64,String?,String?,Int64,String,Int64,String,String


In [23]:
deposits_df = filter(row -> first(row.input, 10) == "0xb214faa5", tornado_traces_df)

,transaction_hash,transaction_index,from_address,to_address,value,input,output,trace_type,call_type,reward_type,gas,gas_used,subtraces,trace_address,error,status,block_timestamp,block_number,block_hash,trace_id
,String,Int64,String,String,Float64,String,String?,String,String,Missing,Int64,Int64?,Int64,String?,String?,Int64,String,Int64,String,String
1,0x14a3923eb7f30311240d6cbe838ac7e0639c309017395b36b130938d9894787c,48,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0xb214faa50e50cde66fa6f1227cab1eeebf0aff07940d861fa4b66d3d0b4c85ce6c76bf7f,0x,call,call,missing,990735,912248,41,2,missing,1,2021-05-22 16:22:08 UTC,12485152,0xe6f35f8a11022e6b42818fff43b16690c7fcb88a7931960f7c22cc35104ebf4b,call_0x14a3923eb7f30311240d6cbe838ac7e0639c309017395b36b130938d9894787c_2
2,0xcb7e6a27cd0e9bf71054b61982b20d12e98885667ee61447d02f7ae1de9ce671,36,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0xb214faa52c7747bcfca9ce0281515638387453bcc49c30f4f0331229e8540ecf2fa59c0a,0x,call,call,missing,1317149,941658,41,2,missing,1,2021-04-26 09:49:49 UTC,12315198,0x067f762a302cbced13ff6165da4d309d646e3abbc7863cc56f98434d5624f8cb,call_0xcb7e6a27cd0e9bf71054b61982b20d12e98885667ee61447d02f7ae1de9ce671_2
3,0xade7cb15e967438c42a9eeede86245195591a4d7f12f327f18231eaa3802ed93,68,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0xb214faa5062e06017fab545360bc7c68684c339acc2c05a16cda23ba05074286ebdb0d3a,0x,call,call,missing,1038629,958810,41,2,missing,1,2021-04-15 21:06:02 UTC,12247016,0xd7ce8779f739c346e2e97e8b3a6f7e51896f3783ba99c1e9a23f84f8d8d21bce,call_0xade7cb15e967438c42a9eeede86245195591a4d7f12f327f18231eaa3802ed93_2
4,0x810b236b3524ec07e464db6da2befd0c70550c312422d605ad38caea2f1dbbe9,93,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0xb214faa5268ebc52aa1ede130d959fc68d276bae6a13376f6ae637ab3330a433af3ef9ec,0x,call,call,missing,990760,912274,41,2,missing,1,2021-05-22 16:10:38 UTC,12485099,0xdd0e09ffa6d2b1e917f4e575c1bcd4e5ff22602b4c7862fdc93719871679eed9,call_0x810b236b3524ec07e464db6da2befd0c70550c312422d605ad38caea2f1dbbe9_2
5,0xd5a46295d99ad3030cc487079bded0fd9b9e7fd56075f49b181a2cb430c9ee41,110,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0xb214faa51f25c47c5a5cff19cd1b5ead8cb9f61756c888fb1c893cac089fd50f90afaf3d,0x,call,call,missing,1008171,904962,41,2,missing,1,2021-07-09 01:34:22 UTC,12790291,0x5965f85e68f6b3618274bdbf912151b01af0fb8bdae61f765b19f6773b4df3d8,call_0xd5a46295d99ad3030cc487079bded0fd9b9e7fd56075f49b181a2cb430c9ee41_2
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮


In [24]:
withdrawls_df = filter(row -> first(row.input, 10) == "0x21a0adb6", tornado_traces_df)

,transaction_hash,transaction_index,from_address,to_address,value,input,output,trace_type,call_type,reward_type,gas,gas_used,subtraces,trace_address,error,status,block_timestamp,block_number,block_hash,trace_id
,String,Int64,String,String,Float64,String,String?,String,String,Missing,Int64,Int64?,Int64,String?,String?,Int64,String,Int64,String,String
1,0xc2c9732f7ea93698d46bafa3dbe0f6febcf799945ce016fe779dd7faa2fdc87f,82,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,1.21151e17,0x21a0adb600000000000000000000000000000000000000000000000000000000000000e015d3d1d30de13ba53da140ef7120f92c7f2b7952a336d34d9db51097dc44bfc80cf98273476befb8881206a0a48af6c63dad87d7960d862d5ad886e615706a9e0000000000000000000000002fe8e10633eae71b503c60180c4190710a132237000000000000000000000000be4d1e137a24af091be80ae58d652279665e3a27000000000000000000000000000000000000000000000000000002f30d20eecb00000000000000000000000000000000000000000000000001ae69bce4dc680000000000000000000000000000000000000000000000000000000000000001000ca661773a0664c7314927e92525a32f0631194b1e73a2873b1bdff4b5b01da82e35c99e3602de30071f80fae1b55170f1c0ab03e505759fedcf2c60d84c53781205853b85e25db939e5f1f429a8ef5a1b58fae8cf10e5e1ef24f671a0743372020923e0bd54bb4ca70ae777397cbb959cb4968c0125512534c77c226de85e2c23e77641570d2dd8a46cd52438dcc0c91ec54471e6ab5f165bb76f9a598828870387d21cd35e5ed1ff4bdb1719577faacdb01ccbafaf9c16726a981d86def97b19808cb12a6b727b65dee94edf7984639cb2986e8a72da0245f137eeac5aff4603f36e0bd6033055c04d4e6805cbec2fc4b6724391a146f10a551921e3161f9a,0x,call,call,missing,500758,447979,4,0,missing,1,2021-10-15 17:05:13 UTC,13423878,0x47ed0032aee2cafd9acdf32317c434f07b537b8400f836e9186687122dd7548c,call_0xc2c9732f7ea93698d46bafa3dbe0f6febcf799945ce016fe779dd7faa2fdc87f_0
2,0x888a45d7f5dd7e289b6ff9091b97294a99c8102273f41f09d5ebfafd22e4a833,441,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0x21a0adb600000000000000000000000000000000000000000000000000000000000000e015d3d1d30de13ba53da140ef7120f92c7f2b7952a336d34d9db51097dc44bfc801eee552023154e3c4cc8306bfcb55c99213fe34b52af4cae9f9beaeb7639e100000000000000000000000002fe8e10633eae71b503c60180c4190710a1322370000000000000000000000002fe8e10633eae71b503c60180c4190710a1322370000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001001f5c90d80840b3f7c919eb08d4fbb39744afa4c29bbe4b998b9354baa86426711095597044243810149e3aa21154e6e02fbaca1a86d85d8511fb2737c37fd0152b3f15980aafb7de7692a9db17f88b11e175713e387754ae97118465a1be2b580c37a205ba1b4d9fba65791e59a13187978b7d49351f731870264a1f8732f6f4207fc85c83e9f779556483dd0237369de809c9877673838ff24eee8ff549ba592648674dfe5856702a86fb8efdc7b86a8116839c80fb8e050a8cb8268c6b3d6127dba4df3935704f7185e8dfe6ba65fc88d0af2be0f587bf94ad759e6852d5222b1196b6cadfb5448b83bd8c1bf8e424cc9e5afef7f3f9373c0f93125b60be97,0x,call,call,missing,608808,375202,2,0,missing,1,2021-10-15 17:47:43 UTC,13424054,0x8bf82deaa17e3b3d8b7a2fe13b21ce0cce4992a64dad4175997ebd1d3087d3df,call_0x888a45d7f5dd7e289b6ff9091b97294a99c8102273f41f09d5ebfafd22e4a833_0
3,0xc54ba9eca305609d92fb0abc264a6cbf0d67a6413bb9c83beba66409722952c2,237,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0x21a0adb600000000000000000000000000000000000000000000000000000000000000e015d3d1d30de13ba53da140ef7120f92c7f2b7952a336d34d9db51097dc44bfc82409c9fcc4cdcf4d831e9fd7c6e676a182920ad13640d556e9f66162d782acdd0000000000000000000000002fe8e10633eae71b503c60180c4190710a1322370000000000000000000000002fe8e10633eae71b503c60180c4190710a13223700000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010028ec4a404db08c0a2641e7ced47382999f46b748b73399a2c833fcc3a541c2c82b7ced21345c66f83d72269fc4fbb2d64326c55eec082dbb3ad

In [29]:
nrow(withdrawls_df) + nrow(deposits_df) == nrow(tornado_traces_df)

true

Now the number is getting closer, but is a little bit bigger than the actual number of deposits (92000)

In [49]:
succesful_deposit_traces = filter(row -> ismissing(row.error), deposits_df)

,transaction_hash,transaction_index,from_address,to_address,value,input,output,trace_type,call_type,reward_type,gas,gas_used,subtraces,trace_address,error,status,block_timestamp,block_number,block_hash,trace_id
,String,Int64,String,String,Float64,String,String?,String,String,Missing,Int64,Int64?,Int64,String?,String?,Int64,String,Int64,String,String
1,0x14a3923eb7f30311240d6cbe838ac7e0639c309017395b36b130938d9894787c,48,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0xb214faa50e50cde66fa6f1227cab1eeebf0aff07940d861fa4b66d3d0b4c85ce6c76bf7f,0x,call,call,missing,990735,912248,41,2,missing,1,2021-05-22 16:22:08 UTC,12485152,0xe6f35f8a11022e6b42818fff43b16690c7fcb88a7931960f7c22cc35104ebf4b,call_0x14a3923eb7f30311240d6cbe838ac7e0639c309017395b36b130938d9894787c_2
2,0xcb7e6a27cd0e9bf71054b61982b20d12e98885667ee61447d02f7ae1de9ce671,36,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0xb214faa52c7747bcfca9ce0281515638387453bcc49c30f4f0331229e8540ecf2fa59c0a,0x,call,call,missing,1317149,941658,41,2,missing,1,2021-04-26 09:49:49 UTC,12315198,0x067f762a302cbced13ff6165da4d309d646e3abbc7863cc56f98434d5624f8cb,call_0xcb7e6a27cd0e9bf71054b61982b20d12e98885667ee61447d02f7ae1de9ce671_2
3,0xade7cb15e967438c42a9eeede86245195591a4d7f12f327f18231eaa3802ed93,68,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0xb214faa5062e06017fab545360bc7c68684c339acc2c05a16cda23ba05074286ebdb0d3a,0x,call,call,missing,1038629,958810,41,2,missing,1,2021-04-15 21:06:02 UTC,12247016,0xd7ce8779f739c346e2e97e8b3a6f7e51896f3783ba99c1e9a23f84f8d8d21bce,call_0xade7cb15e967438c42a9eeede86245195591a4d7f12f327f18231eaa3802ed93_2
4,0x810b236b3524ec07e464db6da2befd0c70550c312422d605ad38caea2f1dbbe9,93,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0xb214faa5268ebc52aa1ede130d959fc68d276bae6a13376f6ae637ab3330a433af3ef9ec,0x,call,call,missing,990760,912274,41,2,missing,1,2021-05-22 16:10:38 UTC,12485099,0xdd0e09ffa6d2b1e917f4e575c1bcd4e5ff22602b4c7862fdc93719871679eed9,call_0x810b236b3524ec07e464db6da2befd0c70550c312422d605ad38caea2f1dbbe9_2
5,0xd5a46295d99ad3030cc487079bded0fd9b9e7fd56075f49b181a2cb430c9ee41,110,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0xb214faa51f25c47c5a5cff19cd1b5ead8cb9f61756c888fb1c893cac089fd50f90afaf3d,0x,call,call,missing,1008171,904962,41,2,missing,1,2021-07-09 01:34:22 UTC,12790291,0x5965f85e68f6b3618274bdbf912151b01af0fb8bdae61f765b19f6773b4df3d8,call_0xd5a46295d99ad3030cc487079bded0fd9b9e7fd56075f49b181a2cb430c9ee41_2
6,0xd9e473beaf1cc1e13d89c130c4c9042eca9782dd9c631a79a8efcbef3f99dad7,29,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0xb214faa5236a1349f63a2f6492ca34cd7ccbd950a900063884966f7c4a805f0feb44e950,0x,call,call,missing,996653,918260,41,2,missing,1,2021-05-22 16:11:11 UTC,12485103,0xcda97b1bf9c87dbda7e142159c70be93587b68a5baa65f8a3748f99ba6063605,call_0xd9e473beaf1cc1e13d89c130c4c9042eca9782dd9c631a79a8efcbef3f99dad7_2
7,0x653331dcf9308307a1a00a1f577bff5dbf67a8ff101d0404f65b49d0c75e3f84,59,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0xb214faa5029432378cfeafc99a7c2548098aa2a09dcecafd427d2c92a71487b9287c550b,0x,call,call,missing,992048,913582,41,2,missing,1,2021-07-09 05:34:39 UTC,12791337,0x22857496ea9eec60842499607be720e53e0ad26d6c258c7a097afbbdc79ff571,call_0x653331dcf9308307a1a00a1f577bff5dbf67a8ff101d0404f65b49d0c75e3f84_2
8,0x16aea278524d01eae766694860267654aa5c04d15147de916dcfc02c41f26779,115,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0xb214faa52c4c1a03d536915f86d1881da4f35befc2fc7520e720df22b093e6bbbe9fab8c,0x,call,call,missing,998460,912196,41,2,missing,1,2021-06-26 22:35:49 UTC,12712338,0x94e5dba3dfecf4a55a9f999a624cad9c2606c064d4d499b8eaef4bb018ef1595,call_0x16aea278524d01eae76669

In [59]:
sort!(combine(groupby(succesful_deposit_traces, :from_address), nrow => :count), :count, rev=true)

,from_address,count
,String,Int64
1,0x722122df12d4e14e13ac3b6895a86e84145b6967,50743
2,0x905b63fff465b9ffbf41dea908ceb12478ec7601,22022
3,0xc62c539a7a6fa03f416f7d284fcc785517a607fb,145
4,0x730c4ae1626d85b9fbe5229ebe9b2f22fe12ca5a,140
5,0x34a17418cec67b82d08cf77a987941f99dc87c6b,115
6,0x6530e61a2cbbb92ce237e64c2ae9f5082d38dacd,83
7,0x54459d826a727393134c929ad1f4bc1e16f6f021,82
8,0x978aad63fe35f7a72573c411c3dff53608aaeac9,74
9,0xbe5e1ba665d397d5304f0f546cead0a1e44f242d,72


In [51]:
filter(row -> row.count>100, combine(groupby(succesful_deposit_traces, :from_address), nrow => :count))

,from_address,count
,String,Int64
1,0x722122df12d4e14e13ac3b6895a86e84145b6967,50743
2,0x730c4ae1626d85b9fbe5229ebe9b2f22fe12ca5a,140
3,0x905b63fff465b9ffbf41dea908ceb12478ec7601,22022
4,0xc62c539a7a6fa03f416f7d284fcc785517a607fb,145
5,0x34a17418cec67b82d08cf77a987941f99dc87c6b,115


Looking for "0x722122df12d4e14e13ac3b6895a86e84145b6967" and "0x905b63fff465b9ffbf41dea908ceb12478ec7601" in etherscan, we find that the first one is the "Tornado.Cash: Proxy" and the second one is "Tornado.Cash: Old Proxy"

In [52]:
combine(groupby(withdrawls_df, :from_address), nrow => :count)

,from_address,count
,String,Int64
1,0x722122df12d4e14e13ac3b6895a86e84145b6967,46822
2,0x33350f9ffe031ca593442a189b3bc7a1ea05b3c1,166
3,0x52faf2849177c43595341bf7a097f27be4a5ea20,163
4,0x49516e20b5692839f877a7e9aa62006a5d02a7b1,853
5,0x7d9240d77dc24d73dd5c46e61f218fa6d2211081,158
6,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,2689
7,0x580e2bd60625f146bc32c75a63dbe0f61810ccda,47
8,0xe3e5e551a6a3d6664325c92833553b6bf83a865a,1612
9,0x7d3bb46c78b0c4949639ce34896bfd875b97ad08,4065


In [55]:
sort!(filter(row -> row.count>100, combine(groupby(withdrawls_df, :from_address), nrow => :count)), :count, rev=true)

,from_address,count
,String,Int64
1,0x722122df12d4e14e13ac3b6895a86e84145b6967,46822
2,0x905b63fff465b9ffbf41dea908ceb12478ec7601,14952
3,0x7d3bb46c78b0c4949639ce34896bfd875b97ad08,4065
4,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,2689
5,0xe3e5e551a6a3d6664325c92833553b6bf83a865a,1612
6,0x41a28335c5075c81502a97cebad597f28728a815,1297
7,0x0b157ac0a65319b0a18aa46b0b0d4497e1074456,1275
8,0x0360b232285a801cbae01c984fb680ba2f465fdc,1011
9,0x49516e20b5692839f877a7e9aa62006a5d02a7b1,853


Again the new and old proxy appears

In [56]:
filter(row -> row.from_address == "0x722122df12d4e14e13ac3b6895a86e84145b6967", deposit_transactions_df)

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price
,String,Int64,Int64,String,String,Float64,Int64,Int64,String,Int64,Int64,Missing,Missing,Int64,String,Int64,String,Int64?,Int64?,Int64?,Int64


In [57]:
filter(row -> row.from_address == "0x905b63fff465b9ffbf41dea908ceb12478ec7601", deposit_transactions_df)

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price
,String,Int64,Int64,String,String,Float64,Int64,Int64,String,Int64,Int64,Missing,Missing,Int64,String,Int64,String,Int64?,Int64?,Int64?,Int64


In [58]:
filter(row -> row.from_address == "0x7d3bb46c78b0c4949639ce34896bfd875b97ad08", deposit_transactions_df)

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price
,String,Int64,Int64,String,String,Float64,Int64,Int64,String,Int64,Int64,Missing,Missing,Int64,String,Int64,String,Int64?,Int64?,Int64?,Int64
1,0xbc91fdd01ed08570befb193e0d25ffd37d104e1be0b02ba7535ab9465077447f,172,25,0x7d3bb46c78b0c4949639ce34896bfd875b97ad08,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,1.0e17,1200000,11000000000,0xb214faa5132b31652746dfb48ff7f1cad56cc5475fb03a405d46bd2488db5bc2823968c7,1781165,972795,missing,missing,1,2020-01-23 14:51:46 UTC,9338569,0xf5277d24dfc9a9e03015850ad27de43891240a8bc195b1d0d83464c0ec52198c,missing,missing,missing,11000000000


Okey, it seems that we dont have any of the transactions that interact with the proxy in our firts two df.
Lets see if we remove the two proxys, we can reconstruct our first two df

In [68]:
sort!(combine(groupby(deposits_df, :from_address), nrow => :count), :count, rev=true)

,from_address,count
,String,Int64
1,0x722122df12d4e14e13ac3b6895a86e84145b6967,50933
2,0x905b63fff465b9ffbf41dea908ceb12478ec7601,22224
3,0xc62c539a7a6fa03f416f7d284fcc785517a607fb,145
4,0x730c4ae1626d85b9fbe5229ebe9b2f22fe12ca5a,141
5,0x34a17418cec67b82d08cf77a987941f99dc87c6b,115
6,0x6530e61a2cbbb92ce237e64c2ae9f5082d38dacd,83
7,0x54459d826a727393134c929ad1f4bc1e16f6f021,82
8,0x978aad63fe35f7a72573c411c3dff53608aaeac9,74
9,0xbe5e1ba665d397d5304f0f546cead0a1e44f242d,72


In [70]:
sum(sort!(combine(groupby(succesful_deposit_traces, :from_address), nrow => :count), :count, rev=true)[3:end, 2])

23950